In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
from nltk.tokenize import word_tokenize
import pickle
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
import import_ipynb
import wikt_def_gen
from statistics import mean
import preprocess_utils
import wikipedia

importing Jupyter notebook from wikt_def_gen.ipynb
importing Jupyter notebook from preprocess_utils.ipynb


In [2]:
# infile = open('/home/jackragless/projects/data/DAIC_GLOGEN/wiki_master_dataframe_removed.pkl','rb')
# input_corpus = pickle.load(infile, encoding='latin1')

In [3]:
# text = input_corpus[5000]['text']

In [4]:
curtext = preprocess_utils.clean_text(wikipedia.page('Australia').content)
curtext

'Australia officially the Commonwealth of Australia is a sovereign country comprising the mainland of the Australian continent the island of Tasmania and numerous smaller islands. It is the largest country in Oceania and the worlds sixth-largest country by total area. The population of 26 million is highly urbanised and heavily concentrated on the eastern seaboard. Australias capital is Canberra and its largest city is Sydney. The countrys other major metropolitan areas are Melbourne Brisbane Perth and Adelaide. Indigenous Australians inhabited the continent for about 65000 years prior to the first arrival of Dutch explorers in the early 17th century who named it New Holland. In 1770 Australias eastern half was claimed by Great Britain and initially settled through penal transportation to the colony of New South Wales from 26 January 1788 a date which became Australias national day. The population grew steadily in subsequent decades and by the time of an 1850s gold rush most of the con

In [5]:
sents_xstopwordsx = []
for sent in nltk.sent_tokenize(curtext):
    temp = []
    for word in sent[:-1].lower().split():
        if word not in stop_words:
            temp.append(word)
    sents_xstopwordsx.append(temp)
    
# arr.append([sent[:-1].lower().split() for sent in nltk.sent_tokenize(text)])
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(sents_xstopwordsx)]

In [7]:
## Train doc2vec model
model = Doc2Vec(tagged_data, vector_size=20, window=2, min_count=1, workers=4, epochs = 100)

In [ ]:
term_to_define = wikt_def_gen.extract('Holden')['noun']

In [13]:
#compare doc to definition sentences + domain
for candidate in term_to_define:
    final_candidate = []
    for cand_word in word_tokenize(preprocess_utils.clean_text_keep_bracket_content(candidate)):
        if cand_word.lower() not in stop_words:
            final_candidate.append(cand_word)
    print(' '.join(final_candidate), '<--->', mean([obj[1] for obj in model.docvecs.most_similar(positive=[model.infer_vector(final_candidate)], topn = len(tagged_data))]))

general brand car automobile originating Australia . <---> 0.9235898885287737


In [9]:
#compare doc to definition domain
for candidate in term_to_define:
    final_candidate = []
    final_candidate = candidate[candidate.find('(')+1:candidate.find(')')].replace(',','').lower().split()
    print(' '.join(final_candidate), '<--->', candidate, '<--->', mean([obj[1] for obj in model.docvecs.most_similar(positive=[model.infer_vector(final_candidate)], topn = len(tagged_data))]))

general <---> (general) A large, flat piece of solid material; a solid object that is large and flat. <---> 0.9326047437755685
general <---> (general) A paving stone; a flagstone. <---> 0.9396786659955978
australia <---> (Australia) A carton containing 24 cans of beer. <---> 0.8950989111473686
general <---> (general) An outside piece taken from a log or timber when sawing it into boards, planks, etc. <---> 0.936625107338554
general <---> (general) A bird, the wryneck. <---> 0.9377583459803932
nautical <---> (nautical) The slack part of a sail. <---> 0.29418923643074535
slang <---> (slang) A large, luxury pre-1980 General Motors vehicle, particularly a Buick, Oldsmobile or Cadillac. <---> 0.1682103034010843
surfing <---> (surfing) A very large wave. <---> 0.9466082144724696
computing <---> (computing) A sequence of 12 adjacent bits, serving as a byte in some computers. <---> -0.05889137503445933
geology <---> (geology) Part of a tectonic plate that is being subducted. <---> 0.9466173637

In [ ]:
###AS EXTENSION COMPARE TEXT TITLE TO DEFINITION